<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Read-in-Data" data-toc-modified-id="Read-in-Data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Read in Data</a></span><ul class="toc-item"><li><span><a href="#Read-in-independent-variables,-join-tickers" data-toc-modified-id="Read-in-independent-variables,-join-tickers-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Read in independent variables, join tickers</a></span></li></ul></li><li><span><a href="#Join-in-the-credit-rating-data" data-toc-modified-id="Join-in-the-credit-rating-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Join in the credit rating data</a></span></li><li><span><a href="#Generate-Model-Data" data-toc-modified-id="Generate-Model-Data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Generate Model Data</a></span></li><li><span><a href="#Generate-Models" data-toc-modified-id="Generate-Models-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Generate Models</a></span><ul class="toc-item"><li><span><a href="#Support-Vector-Machines" data-toc-modified-id="Support-Vector-Machines-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Support Vector Machines</a></span></li><li><span><a href="#Neural-Network" data-toc-modified-id="Neural-Network-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Neural Network</a></span></li></ul></li><li><span><a href="#Run-Models" data-toc-modified-id="Run-Models-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Run Models</a></span></li><li><span><a href="#Save-Models" data-toc-modified-id="Save-Models-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Save Models</a></span></li></ul></div>

In [1]:
import pandas as pd 
import numpy as np 
import plotly.graph_objects as go 
import seaborn as sns 
import matplotlib.pyplot as plt 
from tqdm import tqdm 
import tensorflow as tf

## Read in Data

In [2]:
ids= pd.DataFrame() 
## Read in ticker symbols
for i in range(1, 6): 
    df = pd.read_excel('capiq_data/in_process_ids/ids {}.xlsx'.format(i),
                       engine='openpyxl')[['ID', 'IQID', 'IQ Name']]
    ids = pd.concat([ids, df]) 

## See if there are any duplicates 
print(ids.duplicated().sum()) 
## See if there are any nulls 
print(ids.isna().sum()) 
ids.head() 

0
ID         0
IQID       0
IQ Name    0
dtype: int64


,ID,IQID,IQ Name
0,MMM,IQ289194,3M Company
1,ABT,IQ247483,Abbott Laboratories
2,ABBV,IQ141885706,AbbVie Inc.
3,ABMD,IQ247589,"Abiomed, Inc."
4,ACN,IQ972190,Accenture plc


### Read in independent variables, join tickers

In [3]:
## Join IQID for inds so we can see the Ticker and the name 

ind_df = pd.read_csv('small_df.csv') 

ind_df = ind_df.merge(ids, on = ['IQID', 'IQ Name']) 
ind_df.drop(['Unnamed: 0', 'IQ Name', 'IQID', 
            'quarter'], axis = 1, inplace = True) 

## The data is quarterly, but we need to lok at it annually. 
## This means we have to take the mean of the data for all the quarters 
ind_df = ind_df.groupby(['year', 'ID']).mean().reset_index()
                      
ind_df.head() 

,year,ID,IQ_TOTAL_REV,IQ_NI_CF,IQ_AR,IQ_GP,IQ_TOTAL_ASSETS,IQ_AP,IQ_TOTAL_LIAB,IQ_TOTAL_DEBT,IQ_CASH_FINAN,IQ_TOTAL_EQUITY,IQ_CASH_EQUIV
0,1995,CZR,36.380500,5.98350,0.962750,16.722000,53.898,0.819750,34.04225,30.907500,6.802000,19.85575,1.5305
1,1995,HII,455.500000,7.50000,165.000000,33.500000,690.000,83.000000,554.00000,180.000000,-13.500000,136.00000,1.0000
2,1996,CZR,37.662500,4.61575,3.078750,17.350750,171.045,2.952000,104.33325,124.623250,-2.300500,86.56750,4.1460
3,1996,HII,467.500000,13.75000,210.750000,35.250000,740.500,64.000000,599.00000,352.750000,43.250000,209.50000,0.5000
4,1996,HLT,994.333333,15.00000,133.333333,255.666667,4543.300,270.466667,2559.00000,1655.966667,65.333333,1984.30000,296.6000


## Join in the credit rating data

we want an inner join so we can only keep the companies that we have the credit rating for

In [4]:
## Join in the credit Ratings Data
credit_ratings = pd.read_csv('credit.csv')[['Year', 'TickerSymbol',
                                            'DomesticLTICRSPMthlyAvg']]
credit_ratings.rename({'DomesticLTICRSPMthlyAvg': 'rating'}, inplace = True, axis = 1)  

## Add credit ratings to df  
tot_df = credit_ratings.merge(ind_df, how = 'inner', 
                             left_on = ['Year', 'TickerSymbol'], 
                             right_on = ['year', 'ID'])
tot_df.drop(['Year', 'TickerSymbol'], axis = 1, inplace = True) 


## Normalize by Assets 
tot_df.head()
norm_tot_df = tot_df.drop(['rating', 'year', 'ID', 'IQ_TOTAL_ASSETS'], axis = 1) 
norm_tot_df = norm_tot_df.div(tot_df['IQ_TOTAL_ASSETS'], axis = 0)
tot_df = pd.concat([tot_df[['rating', 'year', 'ID', 'IQ_TOTAL_ASSETS']], 
                        norm_tot_df], axis = 1) 

tot_df.head() 

,rating,year,ID,IQ_TOTAL_ASSETS,IQ_TOTAL_REV,IQ_NI_CF,IQ_AR,IQ_GP,IQ_AP,IQ_TOTAL_LIAB,IQ_TOTAL_DEBT,IQ_CASH_FINAN,IQ_TOTAL_EQUITY,IQ_CASH_EQUIV
0,12.000000,2000,AAL,26213.00,0.185366,0.001793,0.049708,0.056804,0.048335,0.726243,0.239194,0.014687,0.273757,0.003395
1,12.666667,2001,AAL,30054.25,0.157740,-0.014657,0.050276,0.033889,0.051191,0.787243,0.281433,0.035028,0.212757,0.005058
2,14.583333,2002,AAL,31230.25,0.139448,-0.028106,0.047118,0.037640,0.044660,0.913826,0.382025,0.020061,0.086174,0.004451
3,18.250000,2003,AAL,29394.75,0.148326,-0.010444,0.029146,0.034641,0.035670,1.009177,0.461341,0.000510,-0.009177,0.005035
4,18.000000,2004,AAL,29496.25,0.158029,-0.006450,0.030707,0.039259,0.035606,1.009764,0.487706,0.002805,-0.009764,0.004958


In [5]:
tot_df_clean = pd.DataFrame() 



for ticker in tqdm(tot_df['ID'].unique()): 
    
    
    ## small df is all the rows with the ticker, sort by year 
    small_df = tot_df[tot_df['ID'] == ticker].sort_values(by = 'year',
                                                             ascending = True)
    
    
    ## Insert a lead rating column. This is the predictor column, 
    ## as we are trying to predict the credit rating for the next year
    small_df.insert(loc = 0, column = 'lead_rating', 
               value = small_df.rating.shift(-1)) 
    
    
    ## Set the index as the year and the ticker. 
    small_df.set_index(['year', 'ID'], inplace = True) 
    
    ## Take the difference between rows. We are looking to find 
    ## differences in credit rating, so we are going to compare it to 
    ## differences in dependent variables. 
    ## We can then drop the nulls. 
    small_df = small_df.diff().dropna()  
    
    tot_df_clean = pd.concat([tot_df_clean, small_df], axis = 0) 

tot_df_clean.head() 

100%|██████████| 312/312 [00:01<00:00, 216.61it/s]


,,lead_rating,rating,IQ_TOTAL_ASSETS,IQ_TOTAL_REV,IQ_NI_CF,IQ_AR,IQ_GP,IQ_AP,IQ_TOTAL_LIAB,IQ_TOTAL_DEBT,IQ_CASH_FINAN,IQ_TOTAL_EQUITY,IQ_CASH_EQUIV
year,ID,,,,,,,,,,,,,
2001,AAL,1.916667,0.666667,3841.25,-0.027626,-0.016450,0.000568,-0.022915,0.002856,0.061000,0.042238,0.020341,-0.061000,0.001662
2002,AAL,3.666667,1.916667,1176.00,-0.018292,-0.013449,-0.003158,0.003751,-0.006531,0.126582,0.100593,-0.014968,-0.126582,-0.000607
2003,AAL,-0.250000,3.666667,-1835.50,0.008878,0.017662,-0.017971,-0.002999,-0.008991,0.095351,0.079315,-0.019550,-0.095351,0.000584
2004,AAL,0.000000,-0.250000,101.50,0.009703,0.003994,0.001561,0.004619,-0.000063,0.000587,0.026365,0.002295,-0.000587,-0.000077
2005,AAL,-0.583333,0.000000,-98.25,0.018106,-0.000838,0.004899,0.003405,0.002764,0.019753,0.000184,0.001727,-0.019753,-0.000034


## Generate Model Data 

In [6]:
def LabelRating(df_np): 
    ## Convert Rating to categoritcal value 
    lead_rating = df_np.copy() 
    
    lead_rating[lead_rating == 0] = 0
    lead_rating[(lead_rating > 0) ] = 1 
    lead_rating[(lead_rating < 0) ] = -1 

    
    return lead_rating 


## Apply the ratings categorical values 
tot_df_clean[['lead_rating', 
              'rating']] = tot_df_clean[['lead_rating',
                                                        'rating']].apply(LabelRating)
## See value counts in graph form
tot_df_clean.lead_rating.value_counts().to_frame().style.bar()

,lead_rating
0.0,2049
-1.0,642
1.0,524


In [7]:
from sklearn.utils import resample 

## Resample the data

rating_0 = tot_df_clean[tot_df_clean.lead_rating ==0 ]
tot_df_clean_sampled = tot_df_clean[tot_df_clean.lead_rating !=0 ]
rating_0 = resample(rating_0, 
                   replace = True, 
                   n_samples = 650,
                    random_state = 123) 

tot_df_clean = pd.concat([rating_0, 
                          tot_df_clean_sampled], axis = 0) 
tot_df_clean.lead_rating.value_counts().to_frame().style.bar()

,lead_rating
0.0,650
-1.0,642
1.0,524


In [8]:
## Order by Year 
tot_df_clean_year = tot_df_clean.reset_index() 
tot_df_clean_year.sort_values(by = 'year',
                              axis = 0, 
                             inplace = True) 
tot_df_clean_year.set_index(['year', 'ID'],
                            inplace = True) 
tot_df_clean_year.head() 

lead_rating  rating  IQ_TOTAL_ASSETS  IQ_TOTAL_REV  IQ_NI_CF  \
year ID                                                                   
2001 JPM           1.0    -1.0      14452.25000     -0.002409 -0.000409   
     F             1.0     1.0      -3709.25000     -0.042495 -0.004145   
     AKAM          1.0     0.0      -2294.16850      0.068819 -1.117473   
     CHTR          1.0     0.0       1842.98550     -0.000547 -0.001261   
     DOV           0.0     0.0        -49.94725     -0.013956 -0.014696   

              IQ_AR    IQ_GP      IQ_AP  IQ_TOTAL_LIAB  IQ_TOTAL_DEBT  \
year ID                                                                 
2001 JPM   0.000000 -0.002409  0.004809       0.001093      -0.005029   
     F    -0.005853 -0.008635  0.002457       0.020101       0.018932   
     AKAM  0.036566  0.020109  0.061503       0.660347       0.497948   
     CHTR  0.000173  0.003984  0.023845       0.007280       0.048779   
     DOV  -0.019740 -0.014525 -0.008472      -0.020854      -0.027863   

           IQ_CASH_FINAN  IQ_TOTAL_EQUITY  IQ_CASH_EQUIV  
year ID                                                   
2001 JPM       -0.009855        -0.001093      -0.026098  
     F         -0.015464        -0.020101       0.004941  
     AKAM       0.001225        -0.660347       0.164600  
     CHTR       0.004572        -0.007280       0.000449  
     DOV       -0.015357         0.020854       0.010926

In [9]:
## Test and train split for time series
train = tot_df_clean_year.head(
    int(tot_df_clean_year.shape[0] * 0.8 ) 
) 

test = tot_df_clean_year.head(
    int(tot_df_clean_year.shape[0] * 0.2) 
)

## Split into x and y 
def SplitData(df): 
    y = df.lead_rating 
    x = df.drop('lead_rating', axis = 1) 
    
    ## Standardize entire data 
    x = (x  - x.mean()) / x.std() 
    return x, y 

train_x, train_y = SplitData(train) 
test_x, test_y = SplitData(test) 

## Generate Models 
### Support Vector Machines

In [10]:
from sklearn.model_selection import TimeSeriesSplit 
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score


def SVM_Fit(train_x, train_y, kernel,
            params = [10**x for x in np.arange(-1,3,0.9)]): 
    '''Fit the SVM Machine given the kernel type, parameters, 
    data''' 
    
    if kernel == 'linear': 
        parameters = {'C': params} 
    else: 
        parameters = {'C': params, 
                     'gamma': params} 
    
    cv = TimeSeriesSplit(n_splits = 5) 
    
    model = GridSearchCV(estimator = SVC(kernel = kernel), 
                        param_grid = parameters, 
                        cv = cv, 
                        verbose = 1) 
    
    model.fit(train_x, train_y) 
    return model

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

def RandomForestsModel(train_x, train_y): 
    '''Random Forest Model'''
    rf = RandomForestClassifier(random_state = 200) 
    rf.get_params()
    
    # Number of trees in random forest
    n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
    # Number of features to consider at every split
    max_features = ['auto', 'sqrt']
    # Maximum number of levels in tree
    max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
    max_depth.append(None)
    # Minimum number of samples required to split a node
    min_samples_split = [2, 5, 10]
    # Minimum number of samples required at each leaf node
    min_samples_leaf = [1, 2, 4]
    # Method of selecting samples for training each tree
    bootstrap = [True, False]
    # Create the random grid
    random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
    
    cv = TimeSeriesSplit(n_splits = 5) 
    
    rf_random = RandomizedSearchCV(estimator = rf, 
                              param_distributions = random_grid, 
                              n_iter = 100, cv = cv, verbose = 2, 
                              random_state = 200, n_jobs = -1)
    
    rf_random.fit(train_x, train_y) 
    return rf_random 

In [12]:
def Predict(fitted_model, test_x, test_y, 
           name):
    '''Prediction Accuracy'''
    prediction = fitted_model.predict(test_x) 
    score = accuracy_score(prediction, test_y) 
    prediction = pd.DataFrame({'prediction_{}'.format(name): prediction})
    print('The {} Model Score is: {}'.format(name, score)) 
    return prediction, score    

### Neural Network

In [13]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(5)
])

model.stop_training = True

In [14]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

## Run Models

In [15]:
# Category range must go from 0 to 5 instead of -2 to 2
y_train = train_y +2 
y_test = test_y +2

model.fit(train_x.to_numpy(), 
          y_train.to_numpy(),
          epochs=1000)


Train on 1452 samples
Epoch 1/1000
1452/1452 [==============================] - 1s 438us/sample - loss: 1.3782 - accuracy: 0.4435
Epoch 2/1000
1452/1452 [==============================] - 0s 62us/sample - loss: 1.0916 - accuracy: 0.5744
Epoch 3/1000
1452/1452 [==============================] - 0s 63us/sample - loss: 0.9865 - accuracy: 0.5957
Epoch 4/1000
1452/1452 [==============================] - 0s 60us/sample - loss: 0.9390 - accuracy: 0.6260
Epoch 5/1000
1452/1452 [==============================] - 0s 60us/sample - loss: 0.9074 - accuracy: 0.6371
Epoch 6/1000
1452/1452 [==============================] - 0s 62us/sample - loss: 0.8837 - accuracy: 0.6467
Epoch 7/1000
1452/1452 [==============================] - 0s 59us/sample - loss: 0.8608 - accuracy: 0.6515
Epoch 8/1000
1452/1452 [==============================] - 0s 59us/sample - loss: 0.8500 - accuracy: 0.6536
Epoch 9/1000
1452/1452 [==============================] - 0s 61us/sample - loss: 0.8554 - accuracy: 0.6577
Epoch 10/1000


1452/1452 [==============================] - 0s 59us/sample - loss: 0.7559 - accuracy: 0.6894
Epoch 77/1000
1452/1452 [==============================] - 0s 56us/sample - loss: 0.7582 - accuracy: 0.6846
Epoch 78/1000
1452/1452 [==============================] - 0s 56us/sample - loss: 0.7552 - accuracy: 0.6763
Epoch 79/1000
1452/1452 [==============================] - 0s 67us/sample - loss: 0.7498 - accuracy: 0.6963
Epoch 80/1000
1452/1452 [==============================] - 0s 66us/sample - loss: 0.7576 - accuracy: 0.6839
Epoch 81/1000
1452/1452 [==============================] - 0s 63us/sample - loss: 0.7412 - accuracy: 0.6928
Epoch 82/1000
1452/1452 [==============================] - 0s 66us/sample - loss: 0.7539 - accuracy: 0.6860
Epoch 83/1000
1452/1452 [==============================] - 0s 58us/sample - loss: 0.7491 - accuracy: 0.6935
Epoch 84/1000
1452/1452 [==============================] - 0s 57us/sample - loss: 0.7503 - accuracy: 0.6963
Epoch 85/1000
1452/1452 [=================

1452/1452 [==============================] - 0s 58us/sample - loss: 0.7238 - accuracy: 0.6990
Epoch 152/1000
1452/1452 [==============================] - 0s 56us/sample - loss: 0.7240 - accuracy: 0.7087
Epoch 153/1000
1452/1452 [==============================] - 0s 56us/sample - loss: 0.7274 - accuracy: 0.6956
Epoch 154/1000
1452/1452 [==============================] - 0s 56us/sample - loss: 0.7283 - accuracy: 0.6977
Epoch 155/1000
1452/1452 [==============================] - 0s 56us/sample - loss: 0.7266 - accuracy: 0.6997
Epoch 156/1000
1452/1452 [==============================] - 0s 55us/sample - loss: 0.7295 - accuracy: 0.6915
Epoch 157/1000
1452/1452 [==============================] - 0s 56us/sample - loss: 0.7304 - accuracy: 0.6963
Epoch 158/1000
1452/1452 [==============================] - 0s 56us/sample - loss: 0.7246 - accuracy: 0.7004
Epoch 159/1000
1452/1452 [==============================] - 0s 57us/sample - loss: 0.7310 - accuracy: 0.6970
Epoch 160/1000
1452/1452 [========

1452/1452 [==============================] - 0s 58us/sample - loss: 0.7149 - accuracy: 0.6990
Epoch 227/1000
1452/1452 [==============================] - 0s 57us/sample - loss: 0.7163 - accuracy: 0.6983
Epoch 228/1000
1452/1452 [==============================] - 0s 56us/sample - loss: 0.7158 - accuracy: 0.7032
Epoch 229/1000
1452/1452 [==============================] - 0s 56us/sample - loss: 0.7128 - accuracy: 0.7052
Epoch 230/1000
1452/1452 [==============================] - 0s 57us/sample - loss: 0.7060 - accuracy: 0.7039
Epoch 231/1000
1452/1452 [==============================] - 0s 56us/sample - loss: 0.7101 - accuracy: 0.6997
Epoch 232/1000
1452/1452 [==============================] - 0s 57us/sample - loss: 0.7066 - accuracy: 0.6977
Epoch 233/1000
1452/1452 [==============================] - 0s 56us/sample - loss: 0.7259 - accuracy: 0.7025
Epoch 234/1000
1452/1452 [==============================] - 0s 57us/sample - loss: 0.7103 - accuracy: 0.7087
Epoch 235/1000
1452/1452 [========

1452/1452 [==============================] - 0s 58us/sample - loss: 0.7047 - accuracy: 0.7073
Epoch 302/1000
1452/1452 [==============================] - 0s 58us/sample - loss: 0.7156 - accuracy: 0.6977
Epoch 303/1000
1452/1452 [==============================] - 0s 56us/sample - loss: 0.7012 - accuracy: 0.7004
Epoch 304/1000
1452/1452 [==============================] - 0s 56us/sample - loss: 0.6978 - accuracy: 0.7073
Epoch 305/1000
1452/1452 [==============================] - 0s 56us/sample - loss: 0.7017 - accuracy: 0.7073
Epoch 306/1000
1452/1452 [==============================] - 0s 56us/sample - loss: 0.6893 - accuracy: 0.7101
Epoch 307/1000
1452/1452 [==============================] - 0s 57us/sample - loss: 0.6864 - accuracy: 0.7101
Epoch 308/1000
1452/1452 [==============================] - 0s 56us/sample - loss: 0.7084 - accuracy: 0.7087
Epoch 309/1000
1452/1452 [==============================] - 0s 56us/sample - loss: 0.6999 - accuracy: 0.7032
Epoch 310/1000
1452/1452 [========

...

1452/1452 [==============================] - 0s 56us/sample - loss: 0.6581 - accuracy: 0.7362
Epoch 677/1000
1452/1452 [==============================] - 0s 56us/sample - loss: 0.6862 - accuracy: 0.7052
Epoch 678/1000
1452/1452 [==============================] - 0s 55us/sample - loss: 0.6706 - accuracy: 0.7252
Epoch 679/1000
1452/1452 [==============================] - 0s 55us/sample - loss: 0.6901 - accuracy: 0.7073
Epoch 680/1000
1452/1452 [==============================] - 0s 55us/sample - loss: 0.6664 - accuracy: 0.7225
Epoch 681/1000
1452/1452 [==============================] - 0s 56us/sample - loss: 0.6817 - accuracy: 0.7245
Epoch 682/1000
1452/1452 [==============================] - 0s 58us/sample - loss: 0.6863 - accuracy: 0.7156
Epoch 683/1000
1452/1452 [==============================] - 0s 56us/sample - loss: 0.6780 - accuracy: 0.7101
Epoch 684/1000
1452/1452 [==============================] - 0s 56us/sample - loss: 0.6748 - accuracy: 0.7218
Epoch 685/1000
1452/1452 [========

1452/1452 [==============================] - 0s 56us/sample - loss: 0.6646 - accuracy: 0.7245
Epoch 752/1000
1452/1452 [==============================] - 0s 55us/sample - loss: 0.6702 - accuracy: 0.7121
Epoch 753/1000
1452/1452 [==============================] - 0s 56us/sample - loss: 0.6610 - accuracy: 0.7252
Epoch 754/1000
1452/1452 [==============================] - 0s 57us/sample - loss: 0.6801 - accuracy: 0.7190
Epoch 755/1000
1452/1452 [==============================] - 0s 62us/sample - loss: 0.6708 - accuracy: 0.7135
Epoch 756/1000
1452/1452 [==============================] - 0s 67us/sample - loss: 0.6550 - accuracy: 0.7190
Epoch 757/1000
1452/1452 [==============================] - 0s 65us/sample - loss: 0.6638 - accuracy: 0.7149
Epoch 758/1000
1452/1452 [==============================] - 0s 67us/sample - loss: 0.6850 - accuracy: 0.7238
Epoch 759/1000
1452/1452 [==============================] - 0s 63us/sample - loss: 0.6683 - accuracy: 0.7121
Epoch 760/1000
1452/1452 [========

1452/1452 [==============================] - 0s 68us/sample - loss: 0.6867 - accuracy: 0.7211
Epoch 827/1000
1452/1452 [==============================] - 0s 65us/sample - loss: 0.6564 - accuracy: 0.7280
Epoch 828/1000
1452/1452 [==============================] - 0s 64us/sample - loss: 0.6669 - accuracy: 0.7348
Epoch 829/1000
1452/1452 [==============================] - 0s 65us/sample - loss: 0.6737 - accuracy: 0.7176
Epoch 830/1000
1452/1452 [==============================] - 0s 66us/sample - loss: 0.6731 - accuracy: 0.7204
Epoch 831/1000
1452/1452 [==============================] - 0s 66us/sample - loss: 0.6588 - accuracy: 0.7252
Epoch 832/1000
1452/1452 [==============================] - 0s 65us/sample - loss: 0.6837 - accuracy: 0.7052
Epoch 833/1000
1452/1452 [==============================] - 0s 66us/sample - loss: 0.6777 - accuracy: 0.7218
Epoch 834/1000
1452/1452 [==============================] - 0s 66us/sample - loss: 0.6421 - accuracy: 0.7190
Epoch 835/1000
1452/1452 [========

1452/1452 [==============================] - 0s 66us/sample - loss: 0.6706 - accuracy: 0.7390
Epoch 902/1000
1452/1452 [==============================] - 0s 64us/sample - loss: 0.6591 - accuracy: 0.7300
Epoch 903/1000
1452/1452 [==============================] - 0s 64us/sample - loss: 0.6481 - accuracy: 0.7362
Epoch 904/1000
1452/1452 [==============================] - 0s 64us/sample - loss: 0.6487 - accuracy: 0.7300
Epoch 905/1000
1452/1452 [==============================] - 0s 63us/sample - loss: 0.6626 - accuracy: 0.7273
Epoch 906/1000
1452/1452 [==============================] - 0s 62us/sample - loss: 0.6678 - accuracy: 0.7128
Epoch 907/1000
1452/1452 [==============================] - 0s 67us/sample - loss: 0.6785 - accuracy: 0.7142
Epoch 908/1000
1452/1452 [==============================] - 0s 63us/sample - loss: 0.6613 - accuracy: 0.7280
Epoch 909/1000
1452/1452 [==============================] - 0s 61us/sample - loss: 0.6654 - accuracy: 0.7231
Epoch 910/1000
1452/1452 [========

1452/1452 [==============================] - 0s 66us/sample - loss: 0.6759 - accuracy: 0.7149
Epoch 977/1000
1452/1452 [==============================] - 0s 64us/sample - loss: 0.6620 - accuracy: 0.7280
Epoch 978/1000
1452/1452 [==============================] - 0s 64us/sample - loss: 0.6505 - accuracy: 0.7204
Epoch 979/1000
1452/1452 [==============================] - 0s 67us/sample - loss: 0.6533 - accuracy: 0.7259
Epoch 980/1000
1452/1452 [==============================] - 0s 66us/sample - loss: 0.6609 - accuracy: 0.7169
Epoch 981/1000
1452/1452 [==============================] - 0s 61us/sample - loss: 0.6523 - accuracy: 0.7211
Epoch 982/1000
1452/1452 [==============================] - 0s 62us/sample - loss: 0.6559 - accuracy: 0.7314
Epoch 983/1000
1452/1452 [==============================] - 0s 62us/sample - loss: 0.6644 - accuracy: 0.7259
Epoch 984/1000
1452/1452 [==============================] - 0s 68us/sample - loss: 0.6511 - accuracy: 0.7300
Epoch 985/1000
1452/1452 [========

In [16]:
results = model.evaluate(test_x.to_numpy(),  y_test.to_numpy(), verbose=2)
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])
predictions = np.argmax(probability_model.predict(test_x.to_numpy()),axis=1)
confusion_matrix = tf.math.confusion_matrix(y_test, predictions)
# np.argmax(predictions,axis=1)
print(confusion_matrix.numpy())

363/1 - 0s - loss: 0.7057 - accuracy: 0.7576
[[  0   0   0   0]
 [  0  67  14   4]
 [  0  19 116  10]
 [  0  12  29  92]]


In [17]:
## Make predictions
sigmoid = SVM_Fit(train_x, train_y, 'sigmoid') 
rbf = SVM_Fit(train_x, train_y, 'rbf') 
linear = SVM_Fit(train_x, train_y,'linear')
#poly = SVM_Fit(train_x, train_y, 'poly') 
poly = SVC(kernel = 'poly').fit(train_x, train_y)

rf = RandomForestsModel(train_x, train_y) 

Fitting 5 folds for each of 25 candidates, totalling 125 fits
Fitting 5 folds for each of 25 candidates, totalling 125 fits
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits


In [18]:
## SVM Score
sigmoid_predict, sigmoid_score = Predict(sigmoid, test_x, test_y, 'sigmoid') 
lin_predict, lin_score = Predict(linear, test_x, test_y, 'linear') 
poly_predict, poly_score = Predict(poly, test_x, test_y, 'poly') 
rbf_predict, rbf_score = Predict(rbf, test_x, test_y, 'radial')

## TF Score 
tf_score = accuracy_score(predictions, y_test.to_numpy()) 

## Random Forest Score
random_predict, random_score = Predict(rf, test_x, test_y, 
                                      'Random Forest')

The sigmoid Model Score is: 0.699724517906336
The linear Model Score is: 0.7052341597796143
The poly Model Score is: 0.6253443526170799
The radial Model Score is: 0.7327823691460055
The Random Forest Model Score is: 0.7603305785123967


In [19]:
from sklearn.metrics import confusion_matrix
print("Sigmoid",confusion_matrix(test_y,sigmoid_predict,normalize="true"),sep="\n")
print("Linear",confusion_matrix(test_y,lin_predict,normalize="true"),sep="\n")
print("Poly",confusion_matrix(test_y,poly_predict,normalize="true"),sep="\n")
print("Radial",confusion_matrix(test_y,rbf_predict,normalize="true"),sep="\n")
print("Random Forest",confusion_matrix(test_y,random_predict,normalize="true"),sep="\n")

Sigmoid
[[0.55294118 0.36470588 0.08235294]
 [0.02758621 0.86206897 0.11034483]
 [0.05263158 0.33082707 0.61654135]]
Linear
[[0.51764706 0.4        0.08235294]
 [0.0137931  0.87586207 0.11034483]
 [0.04511278 0.31578947 0.63909774]]
Poly
[[0.58823529 0.34117647 0.07058824]
 [0.04137931 0.92413793 0.03448276]
 [0.04511278 0.63157895 0.32330827]]
Radial
[[0.64705882 0.29411765 0.05882353]
 [0.06206897 0.82758621 0.11034483]
 [0.04511278 0.27067669 0.68421053]]
Random Forest
[[0.8        0.14117647 0.05882353]
 [0.10344828 0.80689655 0.08965517]
 [0.09774436 0.21804511 0.68421053]]


In [25]:
fig = go.Figure() 
model_names = ['Sigmoid SVM', 'Radial SVM', 'Linear SVM', 'Polynomial SVM', 'Neural Network']

model_accuracy = [sigmoid_score, rbf_score, lin_score, poly_score, tf_score]

fig.add_trace(go.Bar(x = model_names, 
                    y = model_accuracy, 
                    text = model_accuracy, 
                    textposition = 'auto'))
fig.update_layout(title = 'Model Accuracy Scores Numeric Prediction')

fig.update_yaxes(title_text = 'Accuracy Score') 
fig.update_xaxes(title_text = "Model")
fig.show()

In [ ]:
#!conda install psutil --y
fig.write_image('final_accuracy.jpeg')